In [174]:
import pandas as pd

import numpy as np

from datetime import datetime

import requests
from requests.exceptions import RequestException
import json

from datetime import datetime




IMPORT RELEVANT CSV FILE


In [175]:
# Main CDM Registry project datasource from Database PA POA
df_activities= pd.read_excel('./input/CDM-Activities-November.xlsx')

# Country and geographic identifier file for easier mapping
df_country_code = pd.read_excel('./input/cdm-country-code.xlsx')


# Countries in CADTrust, to check with the mismatch from UNFCCC in later data processing
df_cadt_country = pd.read_excel("./input/CADTrust_Country.xlsx")

# LOAD UNEP file for PDD mapping
df_unep = pd.read_excel("./input/cdm-pipeline.xlsx")







INITIAL CLEANUP


In [177]:

# FILL CDM project reference number NAN value to 0, since there are almost 3000 of them

df_activities['CDM project reference number'] = df_activities['CDM project reference number'].fillna(0).astype(int)


# Check projects with more than one host countries/ "List of host countries (ISO 2)"
more_countries = df_activities[df_activities["List of host countries (ISO 2)"].str.len() > 2]

# Create excel files for projects with more than one countries/ "List of host countries (ISO 2)",
more_countries.to_excel("./output/CDM_projects_many_countries.xlsx")

# Map to CAD Trust based on the first country in the column
df_activities["List of host countries (ISO 2)"] = df_activities["List of host countries (ISO 2)"].str[:2]

# Apply geographic identifier that is suitable for CAD Trust mapping
df_country_code['geographicIdentifier'] = df_country_code.apply(
    lambda row: "{\"latitude\":" + str(row["latitude"]) + ",\"longitude\":" + str(row["longitude"]) + "}", axis=1
)

# Key Value mapping of PDD Consultant for CAD Trust projectDeveloper
pdd_developer_map = df_unep.set_index('Unique project ID')['PDD Consultant'].to_dict()

# Key Value mapping of reference number to unique identifier that will be useful for ISSUANCES and UNITS
id_to_id = df_activities.set_index('CDM project reference number')['Unique project identifier (traceable with Google)'].to_dict()

#Key Value mapping Country Code to CAD Trust Country Field
country_name_map = df_country_code.set_index('country')['cadtcountry'].to_dict()


#Key Value mapping Country Code to Geography Map
geography_map = df_country_code.set_index('name')['geographicIdentifier'].to_dict()

#Key Value mapping verification report date
verification_report_date_map = df_activities.set_index('Unique project identifier (traceable with Google)')['Start of first crediting period'].to_dict()


# Map to status fields from CDM to CAD Trust Project Status Picklist Value
project_status_map = {
    "Registered": "Registered",
    "Rejected": "Withdrawn",
    "Withdrawn":"Withdrawn",
    "WithdrawnBeforePublication": "Withdrawn",
    "Withdrawn Before Publication": "Withdrawn",
    "Deregistered": "De-registered",
    "Pending Publication": "Authorized",
    "Provisional": "Validated",
    "Validation Replaced": "Validated",
    "Requesting Registration": "Listed",
    "Validation Public": "Validated",
    "Validation Terminated": "Withdrawn",
    "NA":"Listed",
    "":"Listed",
    np.nan: "Listed"  # Handling actual NaN values
}
# Map from sector code number to it's description
sector_code_map = {
  "1": 'Energy industries (renewable - / non-renewable sources)',
  "2": 'Energy distribution',
  "3": 'Energy demand',
  "4": 'Manufacturing industries',
  "5": 'Chemical industries',
  "6": 'Construction',
  "7": 'Transport',
  "8": 'Mining/mineral production',
  "9": 'Metal production',
  "10": 'Fugitive emissions from fuels (solid, oil and gas)',
  "11": 'Fugitive emissions from production and consumption of halocarbons and sulphur hexafluoride',
  "12": 'Solvent use',
  "13": 'Waste handling and disposal',
  "14": 'Afforestation and reforestation',
  "15": 'Agriculture'
}


# basic cleanup
def fillna_by_dtype(df):
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):  # Check if the column is numeric
            df[column] = df[column].fillna(0)
        elif pd.api.types.is_object_dtype(df[column]):  # Check if the column is of object type (e.g., strings)
            df[column] = df[column].fillna('NA')
        elif pd.api.types.is_datetime64_any_dtype(df[column]):  # Check if the column is datetime
            df[column] = df[column].fillna(pd.Timestamp.min)
    return df

    


set_cadt_country = set(df_cadt_country['Country'])
set_country_code = set(df_country_code['name'])

# Find non-matching elements
non_matching_in_cadt_country = set_cadt_country - set_country_code
non_matching_in_country_code = set_country_code - set_cadt_country




In [178]:
# Rename the df column to match CADTrust field

rename_columns  = {
  "Unique project identifier (traceable with Google)": "originProjectId",
  "Registration project title": "projectName",
  "Type of CDM project: PA/PoA": "program",
  "Sectoral scope number(s)": "sector",
  "Project type (UNEP DTU)": "projectType",
  "Website project status": "projectStatus",
  "Methodologies used at registration": "methodology",
  "DOE": "validationBody",
  "Start of validation": "validationDate",
  "Country subregion":"inCountryRegion",
  "List of host countries (ISO 2)": "country",

}

df_project = df_activities[rename_columns.keys()].rename(columns=rename_columns)

# Adding other relevant values to CAD Trust field
df_project["registryOfOrigin"]= "CDM Registry"
df_project["currentRegistry"]= "CDM Registry"
df_project['unitMetric'] = 'tCO2e'
df_project['methodology'] = "CDM - " + df_project['methodology']
df_project['projectId'] = df_project['originProjectId']
df_project['projectLink'] = 'https://cdm.unfccc.int/Projects/Validation/DB/' +df_project['projectId']+"/view.html"
df_project['coveredByNDC'] = "Unknown"
df_project['projectStatusDate'] = datetime.now().strftime('%Y-%m-%d')

# Map the countries
df_project['country'] = df_project['country'].map(country_name_map)
df_project['sector'] = df_project['sector'].map(sector_code_map)
df_project['projectDeveloper'] = df_project['projectId'].map(pdd_developer_map)
df_project["geographicIdentifier"]= df_project['country'].map(geography_map)
df_project['description'] = df_project['projectName']
df_project['ndcInformation'] = "NA"

# Implement the fillna function created earlier
df_project = fillna_by_dtype(df_project)
df_project["projectStatus"]= df_project['projectStatus'].map(project_status_map)


projID_to_country = df_project.set_index('projectId')['country'].to_dict()



# Output to check the new DataFrame
df_project.to_excel("./output/CDM-Projects-mapped.xlsx")

# df_project['country'].head(10)





Findings After Failing to Push data to Chia staging API

In [180]:
# Yemen and Cape Verde not in CAD Trust picklist. This require approval from technical committee otherwise we cant push the data
# 5 projects in total
df_project_yemen_cape_verde = df_project[(df_project['country'] == 'Yemen') | (df_project['country'] == 'Cape Verde')]
# df_with_nan_cdmID has One nan value which is project CDM 10121 row 8459

df_with_nan_cdmID = df_project[df_project['projectId']=='NA']

# Projects that have projectID, but the rest are not filled properly
df_with_nan_validation_date = df_project[df_project['validationDate']=='NA']

# Telling this issue to unfccc
df_to_inform_UNFCCC_1 = df_activities[df_activities['Unique project identifier (traceable with Google)']
                                        .isna()]

df_to_inform_UNFCCC_2 = df_activities[df_activities['Unique project identifier (traceable with Google)']
                                        .isin(df_with_nan_validation_date['projectId'])]

# Concatenating the two DataFrame into one
df_to_inform_UNFCCC = pd.concat([df_to_inform_UNFCCC_1, df_to_inform_UNFCCC_2])

df_to_inform_UNFCCC.to_excel("./output/Uncleaned_CDM_Projects.xlsx")



DROP THE ROWS OF DATA MENTIONED ABOVE. AFTER THAT, the API call should be smoothly post all the entire data to CAD Trust chia datalayer local staging mode

In [181]:

# df_cleaned_project = df_project[~((df_project['country'] == 'Yemen') | (df_project['country'] == 'Cape Verde')| (df_project['country'] == 'Bhutan'))]

# Select rows where projectId is not 'NA'
df_cleaned_project = df_project[df_project['projectId'] != 'NA']


df_cleaned_project = df_cleaned_project[
    ~df_project['country'].isin([ 'Bhutan'])]


# Select rows where validationDate is not 'NA'
df_cleaned_project = df_cleaned_project[df_cleaned_project['validationDate'] != 'NA']

df_cleaned_project.to_excel("./output/cdm_projects_mapped_cleaned.xlsx")
# df_cleaned_project
df_bhutan = df_project[df_project['country'] == 'Bhutan']
df_bhutan

df_cleaned_project



/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_3913/2822730733.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_cleaned_project = df_cleaned_project[


,originProjectId,projectName,program,sector,projectType,projectStatus,methodology,validationBody,validationDate,inCountryRegion,...,currentRegistry,unitMetric,projectId,projectLink,coveredByNDC,projectStatusDate,projectDeveloper,geographicIdentifier,description,ndcInformation
0,AuxID,Project for GHG emission reduction by thermal ...,PA,Fugitive emissions from production and consump...,HFCs,Registered,CDM - AM0001,SGS United Kingdom Limited (SGS),2003-12-01 00:00:00,Southern Asia,...,CDM Registry,tCO2e,AuxID,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,2023-12-09,PricewaterhouseCoopers,"{""latitude"":20.593684,""longitude"":78.96288}",Project for GHG emission reduction by thermal ...,NA
1,LJ80OQZR2JVOLTZZX4Y0EGB1N18AVO,HFC Decomposition Project in Ulsan,PA,Fugitive emissions from production and consump...,HFCs,Registered,CDM - AM0001,Japan Quality Assurance Organisation (JQA),2003-12-11 00:00:00,Eastern Asia,...,CDM Registry,tCO2e,LJ80OQZR2JVOLTZZX4Y0EGB1N18AVO,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,2023-12-09,Climate Experts,NA,HFC Decomposition Project in Ulsan,NA
2,SQA16OAS75TKWYZTTKK7G1LO0O8CMU,Brazil NovaGerar Landfill Gas to Energy Project,PA,Waste handling and disposal,Landfill gas,Registered,CDM - AM0003,Det Norske Veritas- CUK,2004-04-05 00:00:00,South America,...,CDM Registry,tCO2e,SQA16OAS75TKWYZTTKK7G1LO0O8CMU,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,2023-12-09,EcoSecurities,"{""latitude"":-14.235004,""longitude"":-51.92528}",Brazil NovaGerar Landfill Gas to Energy Project,NA
3,HG20LFVJPFURJJ4M6E6MS07HTOZTR4,La Esperanza Hydroelectric Project,PA,Energy industries (renewable - / non-renewable...,Hydro,Registered,CDM - AMS-I.D.,Det Norske Veritas- CUK,2004-07-04 00:00:00,Central America,...,CDM Registry,tCO2e,HG20LFVJPFURJJ4M6E6MS07HTOZTR4,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,2023-12-09,"WB-CF, 2E Carbon Access","{""latitude"":15.199999,""longitude"":-86.241905}",La Esperanza Hydroelectric Project,NA
4,V3ZBEVSXGFNT31E2CECR2NYE93FW5L,Project for GHG Emission Reduction by Thermal ...,PA,Fugitive emissions from production and consump...,HFCs,Registered,CDM - AM0001,Japan Quality Assurance Organisation (JQA),2005-12-06 00:00:00,Eastern Asia,...,CDM Registry,tCO2e,V3ZBEVSXGFNT31E2CECR2NYE93FW5L,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,2023-12-09,"WB-CF, SEPA FECO","{""latitude"":35.86166,""longitude"":104.195397}",Project for GHG Emission Reduction by Thermal ...,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13168,QJD9XJ5HBYCRZR6YV9ELX4WLEC5BJ1,Reduction in steam consumption through revampi...,PA,Energy demand,EE Industry,Withdrawn,CDM - AM0018,Bureau Veritas Certification Holding SAS (BVCH),2007-03-21 00:00:00,Southern Asia,...,CDM Registry,tCO2e,QJD9XJ5HBYCRZR6YV9ELX4WLEC5BJ1,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,2023-12-09,Rashtriya Chemicals & Fertilizers,"{""latitude"":20.593684,""longitude"":78.96288}",Reduction in steam consumption through revampi...,NA
13169,4FFA579VM5UA6G3BHJYNRH5Q5Q6S6W,Usina Petribu Renewable Generation with Sugarc...,PA,Energy industries (renewable - / non-renewable...,Biomass Energy,Withdrawn,CDM - ACM0006,SGS United Kingdom Limited (SGS),2006-04-12 00:00:00,South America,...,CDM Registry,tCO2e,4FFA579VM5UA6G3BHJYNRH5Q5Q6S6W,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,2023-12-09,MaxAmbiental,"{""latitude"":-14.235004,""longitude"":-51.92528}",Usina Petribu Renewable Generation with Sugarc...,NA
13170,R33WIRKXKCNOCGHYW84XRF3SJ0KKNI,Power generation from renewable sources  Aiur...,PA,Energy industries (renewable - / non-renewable...,Hydro,Withdrawn,CDM - ACM0002,RINA Services S.p.A. (RINA),2009-05-08 00:00:00,South America,...,CDM Registry,tCO2e,R33WIRKXKCNOCGHYW84XRF3SJ0KKNI,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,2023-12-09,Waycarbon,"{""latitude"":-14.235004,""longitude"":-51.92528}",Power generation from renew

In [182]:

df_issuances = pd.read_excel("./input/CDM-Issuances-November.xlsx")
# df_issuances.drop('DOE', axis=1, inplace=True)
df_issuances.drop('PA/PoA', axis=1, inplace=True)
df_issuances.drop('CP', axis=1, inplace=True)
df_issuances.drop('Monitoring report number', axis=1, inplace=True)
df_issuances.drop('Last_updated', axis=1, inplace=True)
df_issuances.drop('Project type (UNEP DTU)', axis=1, inplace=True)
df_issuances.drop('Project subtype (UNEP DTU)', axis=1, inplace=True)
# df_issuances.drop('Units - Total', axis=1, inplace=True)
df_issuances.drop('Issuance date', axis=1, inplace=True)
df_issuances.drop('HostParty', axis=1, inplace=True)


issuances_columns={
    "CDM project reference number":"projectId",
    "DOE":"verificationBody",
    "Issuance process ID":"id",
    "Monitoring report started":"startDate",
    "Monitoring report ended":"endDate",
     "Units - Total":"unitCount",
}
df_issuances['Monitoring report started'] = pd.to_datetime(df_issuances['Monitoring report started'])
df_issuances['Monitoring report ended'] = pd.to_datetime(df_issuances['Monitoring report ended'])
df_issuances['CDM project reference number'] = df_issuances['CDM project reference number'].map(id_to_id)
df_issuances = df_issuances[issuances_columns.keys()].rename(columns=issuances_columns)


df_issuances['verificationReportDate']=df_issuances['projectId'].map(verification_report_date_map)
df_issuances['verificationApproach']="NA"



df_issuances = fillna_by_dtype(df_issuances)
print(len(df_issuances['projectId'].unique()))
df_issuances


# df_aggregated = df_issuances.groupby('projectId')['unitCount'].sum().reset_index()
df_iss_completed = pd.read_excel("./input/CDM_Issuance_Completed_November.xlsx")
df_iss_completed['Project number'] =df_iss_completed['Project number'].map(id_to_id)

df_iss_completed


def rename_duplicates(df, column_name):
   
    df_copy = df.copy()
    duplic = df_copy.duplicated(subset=column_name, keep=False)
    duplicates = df_copy[duplic].copy()
    duplicates['dup_count'] = duplicates.groupby(column_name).cumcount() + 1
    duplicates[column_name] = duplicates[column_name].astype(str) + '-' + duplicates['dup_count'].astype(str)
    duplicates.drop('dup_count', axis=1, inplace=True)
    df_copy.update(duplicates)
    return df_copy

print(len(df_iss_completed))
df_issuances = rename_duplicates(df_issuances, 'id')

df_issuances= df_issuances.merge(
    df_iss_completed, 
    left_on=['projectId', 'unitCount'], 
    right_on=['Project number', 'Total'],
    how='left'
)

df_issuances.drop('Completion date', axis=1, inplace=True)
df_issuances.drop('Transaction type', axis=1, inplace=True)
df_issuances.drop('Total', axis=1, inplace=True)
df_issuances.drop('Project number', axis=1, inplace=True)
df_issuances.drop('PARTY CODE', axis=1, inplace=True)
df_issuances.drop('CP', axis=1, inplace=True)
df_issuances.drop('Unit Type', axis=1, inplace=True)
df_issuances["Status"] = "Held"
df_issuances.drop_duplicates(subset='id', keep='first', inplace=True)
# df_dup_check = df_issuances[df_issuances.duplicated(subset='id', keep=False)]


# df_dup_check
# df_issuances.to_excel("propercleanissuance.xlsx")


nan_exists = df_issuances['SERIAL RANGE'].isna().any()
nan_exists

df_issuances

3703
12570


,projectId,verificationBody,id,startDate,endDate,unitCount,verificationReportDate,verificationApproach,Status,SERIAL RANGE
0,IBK3RIR7HVE1SZLK7YZMZPZ02UW7TS,KEarth,4KEarth1568615888.93,2017-04-01,2019-07-31,20519.0,2014-01-01,NA,Held,258030584 - 258051102
1,KZPP9BVTCRTYEUSRYB2L0ZNRJUDF2X,KEarth,4KEarth1568616446.5,2017-04-01,2019-07-31,18925.0,2014-05-31,NA,Held,258051103 - 258070027
2,P2UF559LX39J2VDPTZ13PAZUO3DRVV,KEarth,4KEarth1585297661.77,2018-01-01,2019-12-31,24595.0,2011-01-01,NA,Held,265420302 - 265444896
3,AXF2WJ2NOM4CHZGDOJW9S9DINIAQWS,KEarth,4KEarth1596186178.27,2018-01-01,2020-03-21,43219.0,2013-03-22,NA,Held,14955868 - 14999086
4,JOUI5BA4DLQS9MVXDKRLL1HHFVQ4M3,KEarth,4KEarth1599113457.49,2015-04-01,2017-12-31,116637.0,2013-10-10,NA,Held,294645212 - 294761848
...,...,...,...,...,...,...,...,...,...,...
12551,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,No info,sergeyf731618828753.52,2016-09-01,2017-09-30,1231148.0,2013-03-15,NA,Held,279718404 - 280949551
12552,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,No info,sergeyf731618828921.37,2017-10-01,2018-12-08,1128556.0,2013-03-15,NA,Held,280949552 - 282078107
12553,D4CRFLKI60AFK14YEHRDNPY16829DF,No info,sergeyf731650363867.4,2016-10-01,2017-06-30,434350.0,2012-11-01,NA,Held,1167744001 - 1168178350
12554,BN70H0KER9GB6Y0BPJBJ3Q4IBIXK7C,No info,slenzen1552989570.98,2017-01-01,2019-01-31,102514.0,2013-12-27,NA,Held,256330654 - 256433167


In [184]:
df_units = pd.read_excel("./input/CDM-Issuances-November.xlsx")


units_columns={
    "HostParty":"unitOwner",
    "CDM project reference number":"projectId",
    "Issuance process ID":"issuanceId",
    "DOE":"verificationBody",
    "Issuance process ID":"id",
   "Units - Total":"unitCount",
   "Issuance date":"vintageYear",

}

df_units = df_units[units_columns.keys()].rename(columns=units_columns)
df_units["unitOwner"] = df_units["unitOwner"].map(country_name_map)
df_units["projectId"] = df_units["projectId"].map(id_to_id)
df_units["countryJurisdictionOfOwner"] = df_units["unitOwner"]
df_units["inCountryJurisdictionOfOwner"] = ""
df_units["projectLocationId"] = ""
df_units["geographyIdentifier"] = df_units["unitOwner"].map(geography_map)
df_units["correspondingAdjustmentDeclaration"] = "Unknown"
df_units["unitRegistryLink"] = 'https://cdm.unfccc.int/Projects/Validation/DB/' +df_units['projectId']+"/view.html"
df_units["vintageYear"] = df_units["vintageYear"].astype(str).str[:4]
df_units["unitStatus"] = "Held"
df_units["unitType"] = ""
df_units["unitBlockStart"] = ""
df_units["unitBlockEnd"] = ""


df_units = fillna_by_dtype(df_units)

df_units

,unitOwner,projectId,id,verificationBody,unitCount,vintageYear,countryJurisdictionOfOwner,inCountryJurisdictionOfOwner,projectLocationId,geographyIdentifier,correspondingAdjustmentDeclaration,unitRegistryLink,unitStatus,unitType,unitBlockStart,unitBlockEnd
0,India,IBK3RIR7HVE1SZLK7YZMZPZ02UW7TS,4KEarth1568615888.93,KEarth,20519,2020,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,Held,,,
1,India,KZPP9BVTCRTYEUSRYB2L0ZNRJUDF2X,4KEarth1568616446.5,KEarth,18925,2020,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,Held,,,
2,India,P2UF559LX39J2VDPTZ13PAZUO3DRVV,4KEarth1585297661.77,KEarth,24595,2021,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,Held,,,
3,Thailand,AXF2WJ2NOM4CHZGDOJW9S9DINIAQWS,4KEarth1596186178.27,KEarth,43219,2021,Thailand,,,"{""latitude"":15.870032,""longitude"":100.992541}",Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,Held,,,
4,India,JOUI5BA4DLQS9MVXDKRLL1HHFVQ4M3,4KEarth1599113457.49,KEarth,116637,2022,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,Held,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12543,India,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,sergeyf731618828753.52,No info,1231148,2022,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,Held,,,
12544,India,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,sergeyf731618828921.37,No info,1128556,2022,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,Held,,,
12545,China,D4CRFLKI60AFK14YEHRDNPY16829DF,sergeyf731650363867.4,No info,434350,2022,China,,,"{""latitude"":35.86166,""longitude"":104.195397}",Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,Held,,,
12546,India,BN70H0KER9GB6Y0BPJBJ3Q4IBIXK7C,slenzen1552989570.98,No info,102514,2020,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,Held,,,


In [185]:
df_retired = pd.read_excel("./input/CDM_Retired_November.xlsx")
df_retired['PARTY CODE'] = df_retired['Project Identifier'].str.split('-').str[0]
df_retired['Project Identifier'] = df_retired['Project Identifier'].str.split('-').str[1].astype(int)



vintage_year_map= df_units.set_index('projectId')['vintageYear'].to_dict()
issuance_id_map= df_units.set_index('projectId')['id'].to_dict()
verif_body_units_map= df_units.set_index('projectId')['verificationBody'].to_dict()
geog_identifier_units_map = df_units.set_index('projectId')['geographyIdentifier'].to_dict()




# df_retired


retired_columns={
    "Status":"unitStatus",
    "Total":"unitTotal",
    "PARTY CODE":"unitOwner",
    "Project Identifier":"projectId",
    "Serial Range":"Serial Range",
    "Transaction Type":"unitStatus"
}



df_retired = df_retired[retired_columns.keys()].rename(columns=retired_columns)
df_retired[['unitBlockStart', 'unitBlockEnd']] = df_retired['Serial Range'].str.split('-', 1, expand=True)
df_retired['unitBlockStart'] = df_retired['unitBlockStart'].fillna(0).astype(int)
df_retired['unitBlockEnd'] = df_retired['unitBlockEnd'].fillna(0).astype(int)
df_retired.drop('Serial Range', axis=1, inplace=True)
df_retired['projectId']=df_retired['projectId'].map(id_to_id)

df_retired["unitOwner"]= df_retired['unitOwner'].map(country_name_map)
df_retired["countryJurisdictionOfOwner"] = df_retired["unitOwner"]
df_retired['inCountryJurisdictionOfOwner'] = ""


df_retired["unitType"] = ""
df_retired["correspondingAdjustmentDeclaration"] = "Unknown"
df_retired["unitRegistryLink"] = 'https://cdm.unfccc.int/Projects/Validation/DB/' +df_retired['projectId']+"/view.html"


df_needed = df_issuances[['projectId', 'id', 'startDate', 'endDate', 
           'unitCount', 'SERIAL RANGE']]
df_needed[['start', 'end']] = df_needed['SERIAL RANGE'].str.split(' - ', expand=True)

# Convert 'start' and 'end' to integers
df_needed['start'] = df_needed['start'].astype(int)
df_needed['end'] = df_needed['end'].astype(int)
df_needed['yearDiff'] = df_needed['endDate'].dt.year - df_needed['startDate'].dt.year
df_needed['countDiff'] = df_needed['end']-df_needed['start']




# Todo df_retired["vintageYear"]= df_retired['projectId'].map(vintage_year_map)
# Todo df_retired["id"]= df_retired['projectId'].map(issuance_id_map)
df_retired["verificationBody"]= df_retired['projectId'].map(verif_body_units_map)
df_retired['geographyIdentifier']= df_retired['projectId'].map(geog_identifier_units_map)
# Todo df_retired["projectLocationId"] = ""
df_retired = fillna_by_dtype(df_retired)
df_retired["unitStatus"] = "Retired"

# Only 1556 issuance
df_filtered = df_needed[df_needed['startDate'].dt.year >= 2016]

# df_filtered
# len(df_filtered['id'].unique())

df_needed



/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_3913/3119358495.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_needed[['start', 'end']] = df_needed['SERIAL RANGE'].str.split(' - ', expand=True)
/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_3913/3119358495.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_needed[['start', 'end']] = df_needed['SERIAL RANGE'].str.split(' - ', expand=True)
/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_3913/3119358495.py:51: Se

,projectId,id,startDate,endDate,unitCount,SERIAL RANGE,start,end,yearDiff,countDiff
0,IBK3RIR7HVE1SZLK7YZMZPZ02UW7TS,4KEarth1568615888.93,2017-04-01,2019-07-31,20519.0,258030584 - 258051102,258030584,258051102,2,20518
1,KZPP9BVTCRTYEUSRYB2L0ZNRJUDF2X,4KEarth1568616446.5,2017-04-01,2019-07-31,18925.0,258051103 - 258070027,258051103,258070027,2,18924
2,P2UF559LX39J2VDPTZ13PAZUO3DRVV,4KEarth1585297661.77,2018-01-01,2019-12-31,24595.0,265420302 - 265444896,265420302,265444896,1,24594
3,AXF2WJ2NOM4CHZGDOJW9S9DINIAQWS,4KEarth1596186178.27,2018-01-01,2020-03-21,43219.0,14955868 - 14999086,14955868,14999086,2,43218
4,JOUI5BA4DLQS9MVXDKRLL1HHFVQ4M3,4KEarth1599113457.49,2015-04-01,2017-12-31,116637.0,294645212 - 294761848,294645212,294761848,2,116636
...,...,...,...,...,...,...,...,...,...,...
12551,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,sergeyf731618828753.52,2016-09-01,2017-09-30,1231148.0,279718404 - 280949551,279718404,280949551,1,1231147
12552,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,sergeyf731618828921.37,2017-10-01,2018-12-08,1128556.0,280949552 - 282078107,280949552,282078107,1,1128555
12553,D4CRFLKI60AFK14YEHRDNPY16829DF,sergeyf731650363867.4,2016-10-01,2017-06-30,434350.0,1167744001 - 1168178350,1167744001,1168178350,1,434349
12554,BN70H0KER9GB6Y0BPJBJ3Q4IBIXK7C,slenzen1552989570.98,2017-01-01,2019-01-31,102514.0,256330654 - 256433167,256330654,256433167,2,102513


In [186]:
df_retired

,unitStatus,unitTotal,unitOwner,projectId,unitStatus,unitBlockStart,unitBlockEnd,countryJurisdictionOfOwner,inCountryJurisdictionOfOwner,unitType,correspondingAdjustmentDeclaration,unitRegistryLink,verificationBody,geographyIdentifier
0,Retired,5294,Brazil,DWXTGTQLAORUROS9KPVZJUSGI8UK70,Retired,65142364,65147657,Brazil,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,RINA,"{""latitude"":-14.235004,""longitude"":-51.92528}"
1,Retired,5294,Brazil,8J911GX7NLREEIL5L09MKJTI9USERE,Retired,60482913,60488206,Brazil,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,TUV NORD,"{""latitude"":-14.235004,""longitude"":-51.92528}"
2,Retired,5000,Brazil,O5C8ZV4YZTL3001AX5RD5V1OL5OM0A,Retired,57297956,57302955,Brazil,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,DNV,"{""latitude"":-14.235004,""longitude"":-51.92528}"
3,Retired,5000,Brazil,HY2WB5CJ5DZTLBFLQE55ZHT0XK2DO2,Retired,61682096,61687095,Brazil,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,TUV SUD,"{""latitude"":-14.235004,""longitude"":-51.92528}"
4,Retired,5000,Brazil,0IET49KTMORLOP6NX7Q56DF7HXTMPV,Retired,57923146,57928145,Brazil,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,DNV,"{""latitude"":-14.235004,""longitude"":-51.92528}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27741,Retired,3,Chile,L9H0347KVT9F7MAMD49TAYUJLMFHEW,Retired,41951955,41951957,Chile,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,TUV NORD,"{""latitude"":-35.675147,""longitude"":-71.542969}"
27742,Retired,1,China,B90WLUX8QKF2591LN21LEVJ82HOO75,Retired,1185996159,1185996159,China,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,LGAI,"{""latitude"":35.86166,""longitude"":104.195397}"
27743,Retired,186,Malawi,YHNSTOHUC3NLV2APHHB6SCWA61Q46T,Retired,207935,208120,Malawi,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,TUV SUD,"{""latitude"":-13.254308,""longitude"":34.301525}"
27744,Retired,129,India,82ORS4DFFAE3F7JI64VMOSYQEYXGCB,Retired,279636945,279637073,India,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,LGAI,"{""latitude"":20.593684,""longitude"":78.96288}"


In [187]:
df_merged = pd.merge(df_retired, df_needed, on='projectId', how='left', suffixes=('_retired', '_issuance'))

# Filter rows where unitBlockStart and unitBlockEnd are within the start and end range
df_linked = df_merged[(df_merged['unitBlockStart'] >= df_merged['start']) & (df_merged['unitBlockEnd'] <= df_merged['end'])]





def checker_playaround(projectId):

    filtered_braz = df_linked[df_linked['projectId'] == projectId]


    fil_ori = df_retired[df_retired['projectId']== projectId]

    fil_haha = df_needed[df_needed['projectId']== projectId]

    filtered_braz.to_excel("testlook.xlsx")
    print("unique issuance id here is",filtered_braz['id'].unique())
    print("unique issuance id here is",fil_haha['id'].unique())
    print("total length 1",len(fil_ori))
    print("total length 2",len(filtered_braz))




df_linked

df_linked['startDate'] = pd.to_datetime(df_linked['startDate'])
df_linked['endDate'] = pd.to_datetime(df_linked['endDate'])

# Function to calculate vintage year
def calculate_vintage_year(row):
    total_years = row['endDate'].year - row['startDate'].year
    total_years = max(total_years, 1)  # Ensure total_years is at least 1
    units_per_year = row['unitCount'] / total_years
    year_offset = (row['unitBlockStart'] - row['start']) // units_per_year
    return row['startDate'].year + int(year_offset)

# Apply the function to each row
df_linked['vintageYear'] = df_linked.apply(calculate_vintage_year, axis=1)




checker_playaround("B90WLUX8QKF2591LN21LEVJ82HOO75")

df_linked_2016 = df_linked[df_linked['startDate'].dt.year >= 2016]


df_linked.columns

# total 3392 retired units
df_linked_2016

print("Issuances belong to retired units",len(df_linked_2016['id'].unique()))


df_linked_2016

df_linked = df_linked[df_linked['startDate'].dt.year >= 2016]

df_linked

/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_3913/3439304012.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_linked['startDate'] = pd.to_datetime(df_linked['startDate'])
/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_3913/3439304012.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_linked['endDate'] = pd.to_datetime(df_linked['endDate'])


unique issuance id here is ['Applus1654080342.92-2' 'Applus1654080342.92-1']
unique issuance id here is ['Applus1654080342.92-1' 'Applus1654080342.92-2']
total length 1 66
total length 2 66
Issuances belong to retired units 450


/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_3913/3439304012.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_linked['vintageYear'] = df_linked.apply(calculate_vintage_year, axis=1)


,unitStatus,unitTotal,unitOwner,projectId,unitStatus,unitBlockStart,unitBlockEnd,countryJurisdictionOfOwner,inCountryJurisdictionOfOwner,unitType,...,id,startDate,endDate,unitCount,SERIAL RANGE,start,end,yearDiff,countDiff,vintageYear
3815,Retired,192791,Republic of Korea,6TJZFLT03HMWXLX279SK3Q0H4CKONF,Retired,149782321,149975111,Republic of Korea,,,...,KFQ1476160956.86,2016-04-01,2016-09-30,196726.0,149778386 - 149975111,149778386.0,149975111.0,0.0,196725.0,2016
3872,Retired,370927,Republic of Korea,U2ODRRPWUQQ6QTZ7ZILTXXHF5971KB,Retired,159060347,159431273,Republic of Korea,,,...,KFQ1476161673.44,2016-04-01,2016-09-30,378497.0,159052777 - 159431273,159052777.0,159431273.0,0.0,378496.0,2016
4148,Retired,1098,Brazil,SQA16OAS75TKWYZTTKK7G1LO0O8CMU,Retired,122651717,122652814,Brazil,,,...,RWTUV1473949399.83,2016-01-01,2016-08-31,106675.0,122633862 - 122740536,122633862.0,122740536.0,0.0,106674.0,2016
4161,Retired,1317,Brazil,SQA16OAS75TKWYZTTKK7G1LO0O8CMU,Retired,122652815,122654131,Brazil,,,...,RWTUV1473949399.83,2016-01-01,2016-08-31,106675.0,122633862 - 122740536,122633862.0,122740536.0,0.0,106674.0,2016
4335,Retired,161037,Republic of Korea,VTSLJ91XBG74R1697J6HQ1EDA61GFX,Retired,160711145,160872181,Republic of Korea,,,...,Korean Standards Association1481729311.2,2016-04-01,2016-09-30,164323.0,160707859 - 160872181,160707859.0,160872181.0,0.0,164322.0,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130076,Retired,6,Guatemala,WF7W47MLD9F0Y2AFTRY9A41JVLCPJB,Retired,4658969,4658974,Guatemala,,,...,poaiss860189517~GT,2018-09-01,2019-07-31,65738.0,4597048 - 4662785,4597048.0,4662785.0,1.0,65737.0,2018
130208,Retired,50,Malawi,97VY4TYOVDSC3TDCQP09BW9LOLRL7V,Retired,1670920,1670969,Malawi,,,...,poaiss644546618~MW,2018-06-01,2019-05-31,564212.0,1111426 - 1675637,1111426.0,1675637.0,1.0,564211.0,2018
130243,Retired,1,India,3P08PLZ59EJOWYIJ8I18Y3T685HU0P,Retired,241845785,241845785,India,,,...,RINA1490363066.04,2016-01-01,2016-06-21,22820.0,241845329 - 241868148,241845329.0,241868148.0,0.0,22819.0,2016
130288,Retired,6,India,I93UV089KHRIJN8F4UO2OM5PT7CE3I,Retired,301438005,301438010,India,,,...,Applus1635250743.01,2019-08-26,2020-12-31,63635.0,301389094 - 301452728,301389094.0,301452728.0,1.0,63634.0,2019


In [188]:
df_retired_clean = df_linked.copy()


print(df_retired_clean.columns)



columns_df_linked = ['unitStatus', 'unitTotal', 'unitOwner', 'projectId', 'unitStatus',
                     'unitBlockStart', 'unitBlockEnd', 'countryJurisdictionOfOwner',
                     'inCountryJurisdictionOfOwner', 'unitType',
                     'correspondingAdjustmentDeclaration', 'unitRegistryLink',
                     'verificationBody', 'geographyIdentifier', 'id', 'startDate', 'endDate',
                     'unitCount', 'SERIAL RANGE', 'start', 'end', 'yearDiff', 'countDiff',
                     'vintageYear']

# List of columns in df_retired_need
columns_df_retired_need = ['unitStatus', 'unitTotal', 'unitOwner', 'projectId', 'unitStatus',
                           'unitBlockStart', 'unitBlockEnd', 'countryJurisdictionOfOwner',
                           'inCountryJurisdictionOfOwner', 'unitType',
                           'correspondingAdjustmentDeclaration', 'unitRegistryLink',
                           'verificationBody', 'geographyIdentifier', 'vintageYear']

# Columns in df_retired_need but not in df_linked
columns_not_in_linked = set(columns_df_retired_need) - set(columns_df_linked)

# Print the result
print("not in the linke dis", columns_not_in_linked)

# WILL NEED ID LATER
df_retired_clean = df_retired_clean[['unitStatus', 'unitTotal', 'unitOwner', 'projectId', 'unitStatus',
       'unitBlockStart', 'unitBlockEnd', 'countryJurisdictionOfOwner',
       'inCountryJurisdictionOfOwner', 'unitType',
       'correspondingAdjustmentDeclaration', 'unitRegistryLink',
       'verificationBody', 'geographyIdentifier','vintageYear','id']]

df_retired_clean.rename(columns={'unitTotal': 'unitCount'}, inplace=True)

df_retired_clean = df_retired_clean.loc[:,~df_retired_clean.columns.duplicated()]

print(len(df_retired_clean.columns))
df_retired_clean.columns


bhutan_projects = df_project[df_project['country'] == 'Bhutan']
bhutan_project_ids = bhutan_projects['projectId'].tolist()
# Filter df_issuance to remove rows where 'projectId' matches those in bhutan_project_ids
df_retired_clean= df_retired_clean[~df_retired_clean['projectId'].isin(bhutan_project_ids)]


df_retired_clean



Index(['unitStatus', 'unitTotal', 'unitOwner', 'projectId', 'unitStatus',
       'unitBlockStart', 'unitBlockEnd', 'countryJurisdictionOfOwner',
       'inCountryJurisdictionOfOwner', 'unitType',
       'correspondingAdjustmentDeclaration', 'unitRegistryLink',
       'verificationBody', 'geographyIdentifier', 'id', 'startDate', 'endDate',
       'unitCount', 'SERIAL RANGE', 'start', 'end', 'yearDiff', 'countDiff',
       'vintageYear'],
      dtype='object')
not in the linke dis set()
15


,unitStatus,unitCount,unitOwner,projectId,unitBlockStart,unitBlockEnd,countryJurisdictionOfOwner,inCountryJurisdictionOfOwner,unitType,correspondingAdjustmentDeclaration,unitRegistryLink,verificationBody,geographyIdentifier,vintageYear,id
3815,Retired,192791,Republic of Korea,6TJZFLT03HMWXLX279SK3Q0H4CKONF,149782321,149975111,Republic of Korea,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,KFQ,NA,2016,KFQ1476160956.86
3872,Retired,370927,Republic of Korea,U2ODRRPWUQQ6QTZ7ZILTXXHF5971KB,159060347,159431273,Republic of Korea,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,KFQ,NA,2016,KFQ1476161673.44
4148,Retired,1098,Brazil,SQA16OAS75TKWYZTTKK7G1LO0O8CMU,122651717,122652814,Brazil,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,SGS,"{""latitude"":-14.235004,""longitude"":-51.92528}",2016,RWTUV1473949399.83
4161,Retired,1317,Brazil,SQA16OAS75TKWYZTTKK7G1LO0O8CMU,122652815,122654131,Brazil,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,SGS,"{""latitude"":-14.235004,""longitude"":-51.92528}",2016,RWTUV1473949399.83
4335,Retired,161037,Republic of Korea,VTSLJ91XBG74R1697J6HQ1EDA61GFX,160711145,160872181,Republic of Korea,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,KSA,NA,2016,Korean Standards Association1481729311.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130076,Retired,6,Guatemala,WF7W47MLD9F0Y2AFTRY9A41JVLCPJB,4658969,4658974,Guatemala,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,Earthood,"{""latitude"":15.783471,""longitude"":-90.230759}",2018,poaiss860189517~GT
130208,Retired,50,Malawi,97VY4TYOVDSC3TDCQP09BW9LOLRL7V,1670920,1670969,Malawi,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,Earthood,"{""latitude"":-13.254308,""longitude"":34.301525}",2018,poaiss644546618~MW
130243,Retired,1,India,3P08PLZ59EJOWYIJ8I18Y3T685HU0P,241845785,241845785,India,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,RINA,"{""latitude"":20.593684,""longitude"":78.96288}",2016,RINA1490363066.04
130288,Retired,6,India,I93UV089KHRIJN8F4UO2OM5PT7CE3I,301438005,301438010,India,,,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,TUV SUD,"{""latitude"":20.593684,""longitude"":78.96288}",2019,Applus1635250743.01


In [189]:
df_units_linked= df_linked.copy()

print("lenoriginal dfUnitsHeld",len(df_units_linked))

grouped_total = df_units_linked.groupby('id')['unitTotal'].sum().reset_index()
grouped_total.rename(columns={'unitTotal': 'aggregatedUnitTotal'}, inplace=True)

# Step 2: Merge the aggregated data back into df_linked
df_units_held = df_units_linked.merge(grouped_total, on='id', how='left')

# Step 3: Calculate totalHeld
df_units_held['totalHeld'] = df_units_held['unitCount'] - df_units_held['aggregatedUnitTotal']

totalHeld_mapping = df_units_held.set_index('id')['totalHeld'].to_dict()




df_units_held

totalHeld_mapping
duplicate_rows = df_units_held[df_units_held['id'].duplicated()]

df_units_held = df_issuances[df_issuances['startDate'].dt.year >= 2016]



df_units_held['totalHeld'] = df_issuances['id'].map(totalHeld_mapping)
df_units_held['totalHeld'] = df_units_held['totalHeld'].fillna(df_units_held['unitCount'])
# df_see = df_units_held[['id','totalHeld','unitCount']]

df_units_held.drop('unitCount',axis=1,inplace=True)

print("df_units_held column is",df_units_held.columns)



df_units_held.rename(columns={'totalHeld': 'unitCount'}, inplace=True)


# df_units_held.to_excel("df_heldinspect.xlsx")

df_units_held[['unitBlockStart', 'unitBlockEnd']] = df_units_held['SERIAL RANGE'].str.split('-', 1, expand=True)
df_units_held.drop('SERIAL RANGE', axis=1, inplace=True)

df_units_held['unitOwner'] = df_units_held['projectId'].map(projID_to_country)

df_units_held["countryJurisdictionOfOwner"] = df_units_held["unitOwner"]
df_units_held["inCountryJurisdictionOfOwner"] = ""
df_units_held["projectLocationId"] = ""


df_units_held.rename(columns={'Project number': 'projectId'}, inplace=True)
df_units_held["geographyIdentifier"] = df_units_held["unitOwner"].map(geography_map)
df_units_held.rename(columns={'Status': 'unitStatus'}, inplace=True)
df_units_held['unitRegistryLink'] = 'https://cdm.unfccc.int/Projects/Validation/DB/' +df_units_held['projectId']+"/view.html"
df_units_held['correspondingAdjustmentDeclaration'] = "Unknown"
df_units_held['unitType'] =""
df_units_held['vintageYear'] =df_units_held['startDate'].dt.year



columns_to_drop = ['verificationReportDate', 'startDate', 'projectLocationId', 
                   'endDate', 'verificationApproach']

# Dropping the columns
df_units_held = df_units_held.drop(columns=columns_to_drop)

df_units_held.columns


print(set(df_units_held.columns)-set(df_retired_clean.columns))
print(set(df_retired_clean.columns)-set(df_units_held.columns))


print(df_retired_clean.columns)

print(df_units_held.columns)
print(len(df_units_held.columns)-len(df_retired_clean.columns))


df_units_held

# print(df_see['totalHeld'].unique())

df_units_held['vintageYear'].unique()
df_units_held['unitBlockEnd'] = pd.to_numeric(df_units_held['unitBlockEnd'], errors='coerce')
df_units_held['unitCount'] = pd.to_numeric(df_units_held['unitCount'], errors='coerce')

# Update 'unitBlockStart' based on the calculation
df_units_held['unitBlockStart'] = df_units_held['unitBlockEnd'] - df_units_held['unitCount']
bhutan_projects = df_project[df_project['country'] == 'Bhutan']
bhutan_project_ids = bhutan_projects['projectId'].tolist()
# Filter df_issuance to remove rows where 'projectId' matches those in bhutan_project_ids
df_units_held= df_units_held[~df_units_held['projectId'].isin(bhutan_project_ids)]




df_units_held
# df_units_held

lenoriginal dfUnitsHeld 3392
df_units_held column is Index(['projectId', 'verificationBody', 'id', 'startDate', 'endDate',
       'verificationReportDate', 'verificationApproach', 'Status',
       'SERIAL RANGE', 'totalHeld'],
      dtype='object')
set()
set()
Index(['unitStatus', 'unitCount', 'unitOwner', 'projectId', 'unitBlockStart',
       'unitBlockEnd', 'countryJurisdictionOfOwner',
       'inCountryJurisdictionOfOwner', 'unitType',
       'correspondingAdjustmentDeclaration', 'unitRegistryLink',
       'verificationBody', 'geographyIdentifier', 'vintageYear', 'id'],
      dtype='object')
Index(['projectId', 'verificationBody', 'id', 'unitStatus', 'unitCount',
       'unitBlockStart', 'unitBlockEnd', 'unitOwner',
       'countryJurisdictionOfOwner', 'inCountryJurisdictionOfOwner',
       'geographyIdentifier', 'unitRegistryLink',
       'correspondingAdjustmentDeclaration', 'unitType', 'vintageYear'],
      dtype='object')
0


/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_3913/3567138896.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_units_held['totalHeld'] = df_issuances['id'].map(totalHeld_mapping)
/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_3913/3567138896.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_units_held['totalHeld'] = df_units_held['totalHeld'].fillna(df_units_held['unitCount'])
/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_3913/3567138896.py:32: SettingWithC

,projectId,verificationBody,id,unitStatus,unitCount,unitBlockStart,unitBlockEnd,unitOwner,countryJurisdictionOfOwner,inCountryJurisdictionOfOwner,geographyIdentifier,unitRegistryLink,correspondingAdjustmentDeclaration,unitType,vintageYear
0,IBK3RIR7HVE1SZLK7YZMZPZ02UW7TS,KEarth,4KEarth1568615888.93,Held,410.0,2.580507e+08,258051102,India,India,,"{""latitude"":20.593684,""longitude"":78.96288}",https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,,2017
1,KZPP9BVTCRTYEUSRYB2L0ZNRJUDF2X,KEarth,4KEarth1568616446.5,Held,378.0,2.580696e+08,258070027,India,India,,"{""latitude"":20.593684,""longitude"":78.96288}",https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,,2017
2,P2UF559LX39J2VDPTZ13PAZUO3DRVV,KEarth,4KEarth1585297661.77,Held,24595.0,2.654203e+08,265444896,India,India,,"{""latitude"":20.593684,""longitude"":78.96288}",https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,,2018
3,AXF2WJ2NOM4CHZGDOJW9S9DINIAQWS,KEarth,4KEarth1596186178.27,Held,43219.0,1.495587e+07,14999086,Thailand,Thailand,,"{""latitude"":15.870032,""longitude"":100.992541}",https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,,2018
5,E6TEOOEF3MAN4SKPFNYJ3QRQZ69LWR,KEarth,4KEarth1602853695.36,Held,16533.0,2.654449e+08,265461429,India,India,,"{""latitude"":20.593684,""longitude"":78.96288}",https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12551,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,No info,sergeyf731618828753.52,Held,24623.0,2.809249e+08,280949551,India,India,,"{""latitude"":20.593684,""longitude"":78.96288}",https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,,2016
12552,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,No info,sergeyf731618828921.37,Held,22571.0,2.820555e+08,282078107,India,India,,"{""latitude"":20.593684,""longitude"":78.96288}",https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,,2017
12553,D4CRFLKI60AFK14YEHRDNPY16829DF,No info,sergeyf731650363867.4,Held,37420.0,1.168141e+09,1168178350,China,China,,"{""latitude"":35.86166,""longitude"":104.195397}",https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,,2016
12554,BN70H0KER9GB6Y0BPJBJ3Q4IBIXK7C,No info,slenzen1552989570.98,Held,98346.0,2.563348e+08,256433167,India,India,,"{""latitude"":20.593684,""longitude"":78.96288}",https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,,2017


In [190]:
# CHecking qc
df_units_held.to_excel("./output/CDM_Held_Units_Cleaned.xlsx")
df_retired_clean.to_excel("./output/CDM_Retired_Units_Cleaned.xlsx")
# df_issuances.to_excel("./output/CDM_Issuances_Cleaned.xlsx")

df_issuances_cleaned = df_issuances[["verificationBody","projectId","id","startDate","endDate","verificationReportDate","verificationApproach"]]
df_issuances_cleaned =df_issuances_cleaned[df_issuances_cleaned['startDate'].dt.year >= 2016]



bhutan_projects = df_project[df_project['country'] == 'Bhutan']
bhutan_project_ids = bhutan_projects['projectId'].tolist()
# Filter df_issuance to remove rows where 'projectId' matches those in bhutan_project_ids
df_issuances_cleaned= df_issuances_cleaned[~df_issuances_cleaned['projectId'].isin(bhutan_project_ids)]


unique_ids_held = df_units_held['id'].unique()
unique_ids_retired = df_retired_clean['id'].unique()
unique_ids_issuances = df_issuances_cleaned['id'].unique()


df_filtered = df_needed[df_needed['startDate'].dt.year >= 2016]



print(set((unique_ids_retired))-set((unique_ids_held)))
print(set((unique_ids_retired))-set((unique_ids_issuances)))
print(set((unique_ids_issuances))-set((unique_ids_held)))
print(set((unique_ids_held))-set((unique_ids_issuances)))

def convert_timestamp_to_string(x):
    if isinstance(x, pd.Timestamp):
        return x.strftime('%Y-%m-%d %H:%M:%S')
    return x

# Columns that contain Timestamps
timestamp_columns = ['startDate', 'endDate', 'verificationReportDate']

# Apply the conversion to the specific columns

# Apply the conversion to the existing timestamp columns
df_issuances_cleaned[timestamp_columns] = df_issuances_cleaned[timestamp_columns].applymap(convert_timestamp_to_string)
issuances_grouped = df_issuances_cleaned.groupby('projectId').apply(lambda x: x.to_dict(orient='records')).reset_index(name='issuances')

issuances_grouped['issuances'][0]


df_issuances_cleaned

# df_issuances_cleaned

issuances_grouped

df_issuances_cleaned
issuances_grouped

set()
set()
set()
set()


,projectId,issuances
0,04U077ODCZ3UXD7L0SICU3TX02VVRA,"[{'verificationBody': 'TUV NORD', 'projectId':..."
1,07XMNBC94OML35ZALF2I5PQ8NWTZNC,"[{'verificationBody': 'TUV NORD', 'projectId':..."
2,08VRO98X524ZJ4UXBTY8TSL2VKCK90,"[{'verificationBody': 'KFQ', 'projectId': '08V..."
3,0AW2JN932WVQGVQY98JYW0J9Q4L2FC,"[{'verificationBody': 'LGAI', 'projectId': '0A..."
4,0CQ93VYA03J9GMXFUTLWFTBNSBAIWO,"[{'verificationBody': 'TUV NORD', 'projectId':..."
...,...,...
646,ZP5RFO43ELK04YFAEHS825I1P6L1R1,"[{'verificationBody': 'Carbon Check', 'project..."
647,ZQ0WY9LBESZ65QW233KHYHTUYT17KC,"[{'verificationBody': 'LGAI', 'projectId': 'ZQ..."
648,ZQXQGXEQ1VIP7XBAPJCVDH3Q0Y88P2,"[{'verificationBody': 'LGAI', 'projectId': 'ZQ..."
649,ZUOVSKW6TRP5JEUPHLZWVC9E9HGNZD,"[{'verificationBody': 'Earthood', 'projectId':..."


In [221]:

url_project_local = "http://localhost:31310/v1/projects"
url_project_testnet = "http://159.223.67.223:31310/v1/projects"


def datetime_to_string(value):
    if isinstance(value, datetime):
        return value.strftime("%Y-%m-%d")
    return value
# Function to send POST request for each row in the DataFrame
def post_projects(df,start_idx,end_idx,url,type,id_mapping,issuance_mapping):
    # Iterate over each row in the DataFrame
    project_to_uuid_mapping = {}
    i=0
    for index, row in df[start_idx:end_idx].iterrows():
    
        request_body = row.to_dict()
        request_body = {k: datetime_to_string(v) for k, v in row.to_dict().items()}
        request_body['projectLocations'] = [
            {
                "country": request_body.get('country', 'Unknown'),
                "inCountryRegion": request_body.get('inCountryRegion', 'Unknown'),
                "geographicIdentifier":request_body.get('geographicIdentifier',{})  # replace with actual geographicIdentifier if available
            }
        ]

        issuance_ids = []
        issuances_for_api = []

        # Uncomment Later 
        if row.get('issuances') is not None and isinstance(row.get('issuances'), list):
            for issuance in row.get('issuances', []):
                # Store the 'id' for later use
                issuance_id = issuance.get('id')
                issuance_ids.append(issuance_id)
                
                # Remove 'id' from the issuance data sent to the API
                issuance_for_api = {key: value for key, value in issuance.items() if key not in ['id', 'projectId']}
                issuances_for_api.append(issuance_for_api)
            request_body['issuances'] = issuances_for_api
        else:

            request_body.pop('issuances', None)
        # request_body.pop('issuances', None)
        # print(issuances_for_api)
        
        # Remove the keys that are not needed for the POST request
        request_body.pop('country', None)
        request_body.pop('inCountryRegion', None)
        request_body.pop('geographicIdentifier', None)

        # print(request_body)
        # Send POST request
        try:
            if type=="create":
                response = requests.post(url, json=request_body)
                # Check if the request was successful
            if type=="put":
                warehouseId = id_mapping[request_body['projectId']]
                print(warehouseId)
                request_body['warehouseProjectId'] =warehouseId
                response =requests.put(url,json=request_body)
            
            if response.status_code == 200:
                response_data = response.json()
                print(f"Successfully posted project with ID: {row['projectId']}, UUID: {response_data.get('uuid')}")
                
            else:
                print("i is",i)
                print(request_body)
                print(f"Failed to post project with ID: {row['originProjectId']}. Status code: {response.status_code}. Message: {response.json()}")
                break
        

        except RequestException as e:
            print(request_body)
            print(f"Request failed for project with ID: {row['originProjectId']}. Error: {e}")
            break

        i+=1
    return project_to_uuid_mapping
# Call the function to start posting data

def put_project(df,start_idx,end_idx,url):
    i=0
    for index, row in df[start_idx:end_idx].iterrows():
    
        request_body = row.to_dict()
        request_body = {k: datetime_to_string(v) for k, v in row.to_dict().items()}

   
        issuance_ids = []
        issuances_for_api = []

        # Uncomment Later 
        if row.get('issuances') is not None and isinstance(row.get('issuances'), list):
            for issuance in row.get('issuances', []):
                # Store the 'id' for later use
                issuance_id = issuance.get('id')
                issuance_ids.append(issuance_id)
                
                # Remove 'id' from the issuance data sent to the API
                issuance_for_api = {key: value for key, value in issuance.items() if key not in ['orgUid']}
                issuances_for_api.append(issuance_for_api)
            request_body['issuances'] = issuances_for_api
        else:

            request_body.pop('issuances', None)
        request_body['projectLocations'] = [{k: v for k, v in location.items() if k not in ['orgUid', 'warehouseProjectId']} for location in request_body['projectLocations'] ]
        try:
            response =requests.put(url,json=request_body)
            
            if response.status_code == 200:
                response_data = response.json()
                print(f"Successfully posted project with ID: {row['projectId']},")
                
            else:
                print("i is",i)
                print(request_body)
                print(f"Failed to post project with ID: {row['originProjectId']}. Status code: {response.status_code}. Message: {response.json()}")
                break
        
        except RequestException as e:
            print(request_body)
            print(f"Request failed for project with ID: {row['originProjectId']}. Error: {e}")
            break

        i+=1

        
    


merged_project_issuances = pd.merge(df_cleaned_project, issuances_grouped, on='projectId', how='left')

# df_project

url_staging_local = "http://localhost:31310/v1/staging"
url_staging_testnet = "http://159.223.67.223:31310/v1/staging"

def get_projects_staging_uuid(url):
    # url = "http://localhost:31310/v1/staging"

    # Make a GET request
    response = requests.get(url)

    
    # Check the response status code
    if response.status_code == 200:
        # The request was successful, and you can access the response content
        projects = response.json()
        project_id_to_uuid = {}
        project_id_to_issuances={}
        project_id_to_locationId={}
        for project in projects:

            uuid = project['uuid']
            # print(uuid)
            projectId = project['diff']['change'][0]['projectId']
            project_id_to_uuid[projectId] = uuid
            issuances = project['diff']['change'][0].get('issuances', None)

            

        # print(uuid_to_project_id)
        return project_id_to_uuid, project_id_to_issuances
    else:
        # The request failed for some reason
        print(f"Request failed with status code {response.status_code}")


def get_projects_mapping(url):
    # Check if the specified conditions are met
    response = requests.get(url)
    projects = response.json()
    # project_data = response.json()
    project_id_to_warehouseId = {}
    project_id_to_issuances={}
    project_id_to_locationId={}
    duplicate_projects = {}
    project_id_to_locations={}

    for project_data in projects:
        if (project_data["orgUid"] == "fc7e4380bc8ba22f98a655e04b54af3aed122db34bb3357b3a9c3eb0ff7039bc" and
            project_data["registryOfOrigin"] == "CDM Registry"):


            warehouseId = project_data["warehouseProjectId"]
            projectId = project_data["projectId"]
            # Mapping of 'projectId' and 'warehouseProjectId'
            if projectId not in project_id_to_warehouseId:
                project_id_to_warehouseId[projectId] = warehouseId

            else:
                duplicate_projects[projectId] = [project_id_to_warehouseId[projectId],warehouseId]
                project_id_to_warehouseId[projectId] = warehouseId
            
            # Mapping of 'projectId' and 'issuance' if 'issuance' exists
            issuance_ids = []
            issuances_for_api = []
        
            if project_data.get('issuances') is not None and isinstance(project_data.get('issuances'), list):
               
                for issuance in project_data.get('issuances', []):
                    # Store the 'id' for later use
                    issuance_id = issuance.get('id')
                    issuance_ids.append(issuance_id)
                    
                    # Remove 'id' from the issuance data sent to the API
                    issuance_for_api = {key: value for key, value in issuance.items() }
                    issuances_for_api.append(issuance_for_api)

                if issuances_for_api!= []:
                    project_id_to_issuances[projectId] = issuances_for_api

            # Mapping of 'projectId' and 'id' of 'projectLocations'
            locations_mapping = {}
            if project_data.get("projectLocations"):
                # print("lho")
                project_id_to_locationId[projectId]= project_data["projectLocations"][0]["id"]
                project_id_to_locations[projectId]= project_data["projectLocations"]

    return project_id_to_warehouseId, project_id_to_issuances, project_id_to_locationId,duplicate_projects,project_id_to_locations

        





# fail in project num 6142

# check if there is a same project but several uuid

# print(uuid_to_project_id_map)


# merged_project_issuances.to_excel("./output/ProjectsIssuancesCombined.xlsx")


In [209]:
# project_to_uuid_map,project_to_issuances = get_projects_staging_uuid(url_staging_testnet)

# print(len(project_to_uuid_map))


get_projects_url_testnet = url_staging_testnet = "http://159.223.67.223:31310/v1/projects"

project_id_to_warehouseId, issuance_mapping, locations_mapping,duplicate_projects_api,project_id_to_locations= get_projects_mapping(get_projects_url_testnet)


print(len(project_id_to_warehouseId))
print(len(issuance_mapping))
print(len(locations_mapping))
print(len(duplicate_projects_api))


13154
651
13154
0


In [136]:
print(len(project_id_to_warehouseId))
print(len(issuance_mapping))
print(len(locations_mapping))
print(len(duplicate_projects_api))


unique_proj_iss_before = issuances_grouped['projectId'].unique()

print("len unique cleaned Issuances before API",len(unique_proj_iss_before))
print("len cleaned Issuances before API",len(issuances_grouped))
print("len issuance mapping from API)",print(len(issuance_mapping)))

print("diff1",set((unique_proj_iss_before))-set((issuance_mapping)))
print("diff2",set((issuance_mapping))-set((unique_proj_iss_before)))

delta_uploaded_issuances = set((unique_proj_iss_before))-set((issuance_mapping))

for delta in delta_uploaded_issuances:
    print(project_id_to_warehouseId[delta])


project_id_to_warehouseId["HG20LFVJPFURJJ4M6E6MS07HTOZTR4"]

# issuance_mapping
# issuances_grouped.to_excel("asdasd.xlsx")

# len(issuance_mapping['U2ODRRPWUQQ6QTZ7ZILTXXHF5971KB'])
temp = issuances_grouped[issuances_grouped['projectId']== 'HG20LFVJPFURJJ4M6E6MS07HTOZTR4']

temp['issuances']

# this function never been used
def get_warehouseId_and_update_issuances(delta_uploaded_issuances, project_id_to_warehouseId, issuances_grouped, specific_project_id):

    # Print warehouse IDs for each project ID in delta_uploaded_issuances
    for delta in delta_uploaded_issuances:
        if delta in project_id_to_warehouseId:
            print(project_id_to_warehouseId[delta])
        else:
            print(f"Warehouse ID not found for project ID: {delta}")

    # Get 'issuances' data for a specific project ID
    specific_issuances = issuances_grouped[issuances_grouped['projectId'] == specific_project_id]['issuances']
    # print(len(specific_issuances['issuances']))
    return specific_issuances







13154
651
13154
0
len unique cleaned Issuances before API 651
len cleaned Issuances before API 651
651
len issuance mapping from API) None
diff1 set()
diff2 set()
1


In [225]:
first_100_projects = merged_project_issuances[0:100]
first_100_projects
# first_100_projects['projectLocationId'] = first_100_projects['projectId'].map(locations_mapping)
first_100_projects['warehouseProjectId'] = first_100_projects['projectId'].map(project_id_to_warehouseId)
first_100_projects['issuances'] = first_100_projects['projectId'].map(issuance_mapping)
first_100_projects['projectLocations'] = first_100_projects['projectId'].map(project_id_to_locations)

projects_column_to_drop = ['country','inCountryRegion','geographicIdentifier','country','inCountryRegion','geographicIdentifier']

first_100_projects= first_100_projects.drop(columns=projects_column_to_drop)

first_100_projects.to_excel("First_100_projects.xlsx")







# put_project(first_100_projects,10,100,url_project_testnet)




/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_3913/2876006884.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_100_projects['warehouseProjectId'] = first_100_projects['projectId'].map(project_id_to_warehouseId)
/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_3913/2876006884.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_100_projects['issuances'] = first_100_projects['projectId'].map(issuance_mapping)
/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_3913/

In [232]:
# will do 9000 to 1000
# post_projects(merged_project_issuances,13000,14000,url_project_testnet,"create")

Successfully posted project with ID: RUDERLGO0VSCEEA12BDEMJ18D6GC38, UUID: 18e8e27f-bbd7-4121-a061-0563dd948688
Successfully posted project with ID: 9BA066E7P8QO2GGY18B11K48Z46CAO, UUID: 9d2b6453-c5cd-4d8b-ac65-1131f4d3bc5e
Successfully posted project with ID: W2SS66VAM7QHKL0J5B2HVSR64WO755, UUID: fc30ea62-7909-4394-a6b1-ca223aa9a12a
Successfully posted project with ID: SIS7YPVICT6NBFQI7J8R3NMJYYCPI7, UUID: 6f17d27c-f109-4a9d-9d04-37bb2aa5c28a
Successfully posted project with ID: P9KM30JU9ISCZX7O76C1RY4S6PO7YC, UUID: b5e2fef0-a680-49f8-b550-ebd5979a0296
Successfully posted project with ID: U0A0CSRYC2DEDTHR67PF18E4N5BKC8, UUID: f34d0e7c-a318-4b66-8101-18aebffff463
Successfully posted project with ID: LTETSDLLVXGU306ZECFIISO11STV8P, UUID: 63dbbb55-ca1b-46e4-8564-73832d1cb68d
Successfully posted project with ID: HUQOLJOM8VWZZPSHNC7650DOALCOMM, UUID: f7646cd9-5697-4e41-8c16-84d568e0e57e
Successfully posted project with ID: VDFNETY6YGOEBRCCLXC9Y1RR6BOEET, UUID: 0e6ff86d-50db-40c1-abdf-0c5bd

{}

In [164]:


df_retired_clean


merged_issuances_retired_units = pd.merge(df_retired_clean, df_issuances_cleaned, on='id', how='left')

# merged_issuances_units
df_issuances_cleaned
df_issuances_cleaned_unique = df_issuances_cleaned.drop_duplicates(subset='id')

df_issuances_cleaned_unique

duplicates = df_issuances_cleaned_unique.duplicated(subset='id', keep=False)

# Filter the DataFrame to only include the duplicates
df_issuances_duplicates = df_issuances_cleaned_unique[duplicates]

# df_issuances_duplicates.to_excel("./output/duplicateIssuances.xlsx")

merged_issuances_retired_units.drop('projectId_x', axis=1, inplace=True)

merged_issuances_retired_units.drop('verificationBody_x', axis=1, inplace=True)
merged_issuances_retired_units.rename(columns={'projectId_y': 'projectId'}, inplace=True)
merged_issuances_retired_units.rename(columns={'verificationBody_y': 'verificationBody'}, inplace=True)
merged_issuances_retired_units['correspondingAdjustmentStatus']= "Not Applicable"
# merged_issuances_units['warehouseProjectId']= merged_issuances_units['projectId'].map(project_to_uuid_map)


merged_issuances_retired_units.dropna()


tsttst = merged_issuances_retired_units[merged_issuances_retired_units['projectId']=='SQA16OAS75TKWYZTTKK7G1LO0O8CMU']

tsttst



merged_issuances_retired_units['unitRegistryLink']='https://cdm.unfccc.int/Projects/Validation/DB/' +merged_issuances_retired_units['projectId']+"/view.html"
merged_issuances_retired_units['warehouseProjectId'] = merged_issuances_retired_units['projectId'].map(project_id_to_warehouseId)
merged_issuances_retired_units['uploadedIssuance']= merged_issuances_retired_units['projectId'].map(issuance_mapping)
merged_issuances_retired_units['inCountryJurisdictionOfOwner']= merged_issuances_retired_units['unitOwner']

# merged_issuances_retired_units['in']= merged_issuances_retired_units['projectId'].map(issuance_mapping)

merged_issuances_retired_units.rename(columns={'id': 'issuanceOriginalId'}, inplace=True)
merged_issuances_retired_units['projectLocationId'] = merged_issuances_retired_units['projectId'].map(project_id_to_warehouseId)

def match_uploaded_issuance(row):
    for uploaded in row['uploadedIssuance']:

        start_date = pd.to_datetime(row['startDate']).strftime('%Y-%m-%dT%H:%M:%S.000Z')
        end_date = pd.to_datetime(row['endDate']).strftime('%Y-%m-%dT%H:%M:%S.000Z')

        # Now compare dates in the same format
        if (uploaded['verificationApproach'] == row['verificationApproach'] and
            uploaded['startDate'] == start_date and
            uploaded['verificationBody'] == row['verificationBody'] and
            uploaded['endDate'] == end_date):
            # print(uploaded)

            matched_issuance = uploaded.copy()
            matched_issuance.pop('orgUid', None) 
            return uploaded['id'], matched_issuance
    
        # print(f"No match for row: {row['verificationApproach']} | {row['startDate']} | {row['verificationBody']} | {row['endDate']}")
        # print(f"Compared against: {uploaded['verificationApproach']} | {uploaded['startDate']} | {uploaded['verificationBody']} | {uploaded['endDate']}")
    
    return None,None # Return None or appropriate value if no match found

merged_issuances_retired_units[['newId', 'matchedIssuance']] = merged_issuances_retired_units.apply(
    lambda row: match_uploaded_issuance(row), 
    axis=1, 
    result_type='expand'
)

null_count = merged_issuances_retired_units['newId'].isna().sum()

# Print the count
print(f"The 'newId' column has {null_count} null values.")
assert null_count == 0, f"There are {null_count} null values in the 'newId' column, but there should be none."


merged_issuances_retired_units.drop('projectId', axis=1, inplace=True)
merged_issuances_retired_units.drop('geographyIdentifier', axis=1, inplace=True)
merged_issuances_retired_units['unitType']="Not Determined"
merged_issuances_retired_units.drop('uploadedIssuance', axis=1, inplace=True)
merged_issuances_retired_units.drop('warehouseProjectId', axis=1, inplace=True)
columns_to_drop = ['verificationBody', 'startDate', 'endDate', 'verificationReportDate', 'verificationApproach','issuanceOriginalId','newId']

# Drop the specified columns
merged_issuances_retired_units = merged_issuances_retired_units.drop(columns=columns_to_drop)
merged_issuances_retired_units.rename(columns={'matchedIssuance': 'issuance'}, inplace=True)
merged_issuances_retired_units['unitBlockStart'] = merged_issuances_retired_units['unitBlockStart'].astype(str)
merged_issuances_retired_units['unitBlockEnd'] = merged_issuances_retired_units['unitBlockEnd'].astype(str)


merged_issuances_retired_units.to_excel("./output/retired_units_api_upload.xlsx")


merged_issuances_retired_units

The 'newId' column has 0 null values.


,unitStatus,unitCount,unitOwner,unitBlockStart,unitBlockEnd,countryJurisdictionOfOwner,inCountryJurisdictionOfOwner,unitType,correspondingAdjustmentDeclaration,unitRegistryLink,vintageYear,correspondingAdjustmentStatus,projectLocationId,issuance
0,Retired,192791,Republic of Korea,149782321,149975111,Republic of Korea,Republic of Korea,Not Determined,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,2016,Not Applicable,be305f42-4951-4a36-b16d-aaa1d35cc560,"{'id': '54a00c62-c363-4e14-8ad9-17f4c9ee113e',..."
1,Retired,370927,Republic of Korea,159060347,159431273,Republic of Korea,Republic of Korea,Not Determined,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,2016,Not Applicable,2beb8d2d-401e-4027-a80e-44c426e929b5,"{'id': 'a9c41fef-24aa-42f7-86d4-b3d54033cab2',..."
2,Retired,1098,Brazil,122651717,122652814,Brazil,Brazil,Not Determined,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,2016,Not Applicable,0402edd8-7c35-49fe-a55d-f2c54d2434ea,"{'id': 'bce642c9-b6a0-46b7-9f72-00d2d27ed2c3',..."
3,Retired,1317,Brazil,122652815,122654131,Brazil,Brazil,Not Determined,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,2016,Not Applicable,0402edd8-7c35-49fe-a55d-f2c54d2434ea,"{'id': 'bce642c9-b6a0-46b7-9f72-00d2d27ed2c3',..."
4,Retired,161037,Republic of Korea,160711145,160872181,Republic of Korea,Republic of Korea,Not Determined,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,2016,Not Applicable,704bc389-66b4-4d19-8cf5-7e83f363dcda,"{'id': 'b481f138-3686-4d95-887f-7fcaa7a16cef',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3385,Retired,6,Guatemala,4658969,4658974,Guatemala,Guatemala,Not Determined,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,2018,Not Applicable,b5c3cddc-be29-47e7-9fc3-f495f19eeefb,"{'id': '603e96ff-cf10-4b69-9f03-1bf9af37e1e2',..."
3386,Retired,50,Malawi,1670920,1670969,Malawi,Malawi,Not Determined,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,2018,Not Applicable,4557cc52-837a-4aa7-a2d7-d946c17cfe7a,"{'id': '0e43a62d-a787-4e82-a3a7-92fb0cebace1',..."
3387,Retired,1,India,241845785,241845785,India,India,Not Determined,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,2016,Not Applicable,c6051e82-b4c0-4b23-9593-b760b4a734a6,"{'id': 'b7e3b5ae-6c15-4418-96c8-89fb8c42b0c0',..."
3388,Retired,6,India,301438005,301438010,India,India,Not Determined,Unknown,https://cdm.unfccc.int/Projects/Validation/DB/...,2019,Not Applicable,baea012e-4376-4c4a-84ac-21dc76d3314b,"{'id': '3dfa5122-9805-483a-9370-c9a6d6ed5fc3',..."


In [168]:
df_units_held


merged_issuances_held_units = pd.merge(df_units_held, df_issuances_cleaned, on='id', how='left')

# merged_issuances_units
df_issuances_cleaned
df_issuances_cleaned_unique = df_issuances_cleaned.drop_duplicates(subset='id')

df_issuances_cleaned_unique

duplicates = df_issuances_cleaned_unique.duplicated(subset='id', keep=False)

# Filter the DataFrame to only include the duplicates
df_issuances_duplicates = df_issuances_cleaned_unique[duplicates]

# df_issuances_duplicates.to_excel("./output/duplicateIssuances.xlsx")

merged_issuances_held_units .drop('projectId_x', axis=1, inplace=True)

merged_issuances_held_units.drop('verificationBody_x', axis=1, inplace=True)
merged_issuances_held_units.rename(columns={'projectId_y': 'projectId'}, inplace=True)
merged_issuances_held_units.rename(columns={'verificationBody_y': 'verificationBody'}, inplace=True)
merged_issuances_held_units['correspondingAdjustmentStatus']= "Not Applicable"
merged_issuances_held_units['correspondingAdjustmentStatus']= "Not Applicable"
# merged_issuances_units['warehouseProjectId']= merged_issuances_units['projectId'].map(project_to_uuid_map)


# merged_issuances_held_units.dropna()


merged_issuances_held_units['unitRegistryLink']='https://cdm.unfccc.int/Projects/Validation/DB/' +merged_issuances_held_units['projectId']+"/view.html"
merged_issuances_held_units['warehouseProjectId'] = merged_issuances_held_units['projectId'].map(project_id_to_warehouseId)
merged_issuances_held_units['uploadedIssuance']= merged_issuances_held_units['projectId'].map(issuance_mapping)
merged_issuances_held_units['inCountryJurisdictionOfOwner']= merged_issuances_held_units['unitOwner']

# merged_issuances_retired_units['in']= merged_issuances_retired_units['projectId'].map(issuance_mapping)

merged_issuances_held_units.rename(columns={'id': 'issuanceOriginalId'}, inplace=True)
merged_issuances_held_units['projectLocationId'] = merged_issuances_held_units['projectId'].map(project_id_to_warehouseId)

def match_uploaded_issuance(row):
    for uploaded in row['uploadedIssuance']:

        start_date = pd.to_datetime(row['startDate']).strftime('%Y-%m-%dT%H:%M:%S.000Z')
        end_date = pd.to_datetime(row['endDate']).strftime('%Y-%m-%dT%H:%M:%S.000Z')

        # Now compare dates in the same format
        if (uploaded['verificationApproach'] == row['verificationApproach'] and
            uploaded['startDate'] == start_date and
            uploaded['verificationBody'] == row['verificationBody'] and
            uploaded['endDate'] == end_date):
            # print(uploaded)

            matched_issuance = uploaded.copy()
            matched_issuance.pop('orgUid', None) 
            return uploaded['id'], matched_issuance
    
        # print(f"No match for row: {row['verificationApproach']} | {row['startDate']} | {row['verificationBody']} | {row['endDate']}")
        # print(f"Compared against: {uploaded['verificationApproach']} | {uploaded['startDate']} | {uploaded['verificationBody']} | {uploaded['endDate']}")
    
    return None,None # Return None or appropriate value if no match found

merged_issuances_held_units[['newId', 'matchedIssuance']] = merged_issuances_held_units.apply(
    lambda row: match_uploaded_issuance(row), 
    axis=1, 
    result_type='expand'
)

null_count = merged_issuances_held_units['newId'].isna().sum()

# Print the count
print(f"The 'newId' column has {null_count} null values.")
assert null_count == 0, f"There are {null_count} null values in the 'newId' column, but there should be none."


merged_issuances_held_units.drop('projectId', axis=1, inplace=True)
merged_issuances_held_units.drop('geographyIdentifier', axis=1, inplace=True)
merged_issuances_held_units['unitType']="Not Determined"
merged_issuances_held_units.drop('uploadedIssuance', axis=1, inplace=True)
merged_issuances_held_units.drop('warehouseProjectId', axis=1, inplace=True)
columns_to_drop = ['verificationBody', 'startDate', 'endDate', 'verificationReportDate', 'verificationApproach','issuanceOriginalId','newId']

# Drop the specified columns
merged_issuances_held_units = merged_issuances_held_units.drop(columns=columns_to_drop)
merged_issuances_held_units.rename(columns={'matchedIssuance': 'issuance'}, inplace=True)
merged_issuances_held_units['unitBlockStart'] = merged_issuances_held_units['unitBlockStart'].astype(str)
merged_issuances_held_units['unitBlockEnd'] = merged_issuances_held_units['unitBlockEnd'].astype(str)




merged_issuances_held_units.to_excel("./output/held_units_api_upload.xlsx")

merged_issuances_held_units

merged_issuances_held_units

The 'newId' column has 0 null values.


,unitStatus,unitCount,unitBlockStart,unitBlockEnd,unitOwner,countryJurisdictionOfOwner,inCountryJurisdictionOfOwner,unitRegistryLink,correspondingAdjustmentDeclaration,unitType,vintageYear,correspondingAdjustmentStatus,projectLocationId,issuance
0,Held,410.0,258050692.0,258051102,India,India,India,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Determined,2017,Not Applicable,44528061-55b1-45d0-bd08-875992bc0ebc,"{'id': 'fbc12764-00d3-4a11-a6d8-4b23d99ff866',..."
1,Held,378.0,258069649.0,258070027,India,India,India,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Determined,2017,Not Applicable,6eff2d36-0dc2-4a11-811a-4a0790594426,"{'id': '5cc2fa9a-1a5e-4d88-be76-e22d0fb8fc9d',..."
2,Held,24595.0,265420301.0,265444896,India,India,India,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Determined,2018,Not Applicable,d4ebe6f0-2e65-4341-8759-38e591e3c369,"{'id': 'ff0f3ff9-6d84-4c81-b0f9-98181aff63e5',..."
3,Held,43219.0,14955867.0,14999086,Thailand,Thailand,Thailand,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Determined,2018,Not Applicable,ae22f935-3659-4cf5-999a-6d637003697a,"{'id': 'bbebd94b-6a8a-4a56-9bc9-b934da481219',..."
4,Held,16533.0,265444896.0,265461429,India,India,India,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Determined,2019,Not Applicable,49981064-bde4-4f01-920c-66166f77305b,"{'id': 'b5f6a546-0eb8-4b6d-8021-aa91421eb7a0',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541,Held,24623.0,280924928.0,280949551,India,India,India,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Determined,2016,Not Applicable,f00be2d1-5f7c-4b30-84d8-1f87d819e1e7,"{'id': '69119ef1-e84c-4f07-a501-338b31720666',..."
1542,Held,22571.0,282055536.0,282078107,India,India,India,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Determined,2017,Not Applicable,f00be2d1-5f7c-4b30-84d8-1f87d819e1e7,"{'id': 'b899cc10-0f1d-4417-8089-aa5ba1e7ebad',..."
1543,Held,37420.0,1168140930.0,1168178350,China,China,China,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Determined,2016,Not Applicable,beddda47-2cf9-40f1-9352-e4fafb24c538,"{'id': '5cbd0463-f688-446a-95b6-d36997051a6f',..."
1544,Held,98346.0,256334821.0,256433167,India,India,India,https://cdm.unfccc.int/Projects/Validation/DB/...,Unknown,Not Determined,2017,Not Applicable,c732c1f7-6878-48b3-9d5c-b9c1167f27b8,"{'id': '1ee4c8b4-0487-48d3-9efd-2a9d1fbd4160',..."


In [162]:
def post_units(df, url, start_index, end_index):
  
    uuids = []
    # Iterate over the specified range of rows
    for i in range(start_index, min(end_index, len(df))):
        row = df.iloc[i]
        data = row.to_dict()
        response = requests.post(url, json=data)

        try:

            if response.status_code == 200:  # Check if the request was successful
                print("i is",i)
                response_data = response.json()
                uuids.append(response_data.get('uuid'))  # Get the UUID and add to the list
            else:
                    print("i is",i)
                    print(data)
                    print(f"Failed to post project with ID: {row['originProjectId']}. Status code: {response.status_code}. Message: {response.json()}")
                    break
            # if type=="update":

            #     response = requests.put(url, json=request_body)
        except RequestException as e:
            print(data)
            print(f"Request failed for project with ID: {row['originProjectId']}. Error: {e}")
            break
    return uuids

# Example usage


In [173]:
url = "http://159.223.67.223:31310/v1/units"  # Replace with your actual URL
start_index = 1000# Starting index
end_index = 2000  # Ending index

# Call the function with the DataFrame and the specified range
uuids = post_units(merged_issuances_held_units, url, start_index, end_index)
uuids

i is 1000
i is 1001
i is 1002
i is 1003
i is 1004
i is 1005
i is 1006
i is 1007
i is 1008
i is 1009
i is 1010
i is 1011
i is 1012
i is 1013
i is 1014
i is 1015
i is 1016
i is 1017
i is 1018
i is 1019
i is 1020
i is 1021
i is 1022
i is 1023
i is 1024
i is 1025
i is 1026
i is 1027
i is 1028
i is 1029
i is 1030
i is 1031
i is 1032
i is 1033
i is 1034
i is 1035
i is 1036
i is 1037
i is 1038
i is 1039
i is 1040
i is 1041
i is 1042
i is 1043
i is 1044
i is 1045
i is 1046
i is 1047
i is 1048
i is 1049
i is 1050
i is 1051
i is 1052
i is 1053
i is 1054
i is 1055
i is 1056
i is 1057
i is 1058
i is 1059
i is 1060
i is 1061
i is 1062
i is 1063
i is 1064
i is 1065
i is 1066
i is 1067
i is 1068
i is 1069
i is 1070
i is 1071
i is 1072
i is 1073
i is 1074
i is 1075
i is 1076
i is 1077
i is 1078
i is 1079
i is 1080
i is 1081
i is 1082
i is 1083
i is 1084
i is 1085
i is 1086
i is 1087
i is 1088
i is 1089
i is 1090
i is 1091
i is 1092
i is 1093
i is 1094
i is 1095
i is 1096
i is 1097
i is 1098
i is 1099


['1e42a397-3c2b-4f35-a9a2-0aa5e3ed1260',
 '9854942e-e0ed-4df0-a3c4-23e4eee4857c',
 '3f48f8e2-50d5-48fa-9a55-9ce6139bae28',
 '37e20ebd-b2e6-42b1-8cad-07cb4cd5aae1',
 '57edeaa3-1adc-44cb-a687-956d96d6317f',
 '84cd5f6b-5acf-415a-a197-8edd40295288',
 '6965c3b2-3679-485d-ad74-4ea85eab580e',
 'a3948923-2665-4812-b872-97d543d88591',
 '9765ff3d-795f-4da2-9386-3e82e260a2f9',
 '121e93ed-be44-4790-88b2-633e06ae2663',
 'e56ecae0-cdd5-45d2-96f7-b744b859cf1d',
 '5b5ee63e-51a9-46d5-acea-99f15fdad348',
 '1850f282-9a37-4784-a234-2b1cc6a524ac',
 '500e207a-cabf-4ef6-86e7-232662330385',
 '4b28e008-3afe-41b6-91e9-50b40b373b6c',
 '2994f35e-9778-4cea-a3dd-758b024eabb3',
 '90ef6fc0-42f2-446e-87fa-9d3639e4f183',
 '4236e1d5-ff0b-4368-8c81-253bcc9b22a8',
 '51e84f9a-0a90-4d6e-9f41-184a27c0dc99',
 'c241c2e3-7caf-4b7a-babd-767583b976ad',
 'ea2e746f-1213-473c-997b-0034aac9c94b',
 '71f507cc-329b-4ef6-8648-cf09511c932b',
 'a7793409-e4a2-498f-8143-3953e3cffb4a',
 '3393f7ed-0b80-4128-a5af-21f510681a43',
 'cf54b72b-2cb1-

In [100]:
df_checking_iss = pd.read_excel("./input/CDM-Issuances-November.xlsx")

df_checking_iss

duplic = df_checking_iss.duplicated(subset='Issuance process ID', keep=False)

# Filter the DataFrame to only include the duplicates
df_duplic = df_checking_iss[duplic]
df_duplic = df_duplic.sort_values(by='CDM project reference number', ascending=True)

df_duplic.to_excel("duplicduplic.xlsx")

len(df_checking_iss)




12548

In [ ]:
def rename_duplicates(df, column_name):
   
    df_copy = df.copy()
    duplic = df_copy.duplicated(subset=column_name, keep=False)
    duplicates = df_copy[duplic].copy()
    duplicates['dup_count'] = duplicates.groupby(column_name).cumcount() + 1
    duplicates[column_name] = duplicates[column_name].astype(str) + '-' + duplicates['dup_count'].astype(str)
    duplicates.drop('dup_count', axis=1, inplace=True)
    df_copy.update(duplicates)
    return df_copy
updated_df = rename_duplicates(df_checking_iss, 'Issuance process ID')
updated_df



In [171]:
# {
#     "message": "Data Validation error",
#     "errors": [
#         "\"inCountryJurisdictionOfOwner\" is not allowed to be empty",
#         "\"unitBlockStart\" must be a string",
#         "\"unitBlockEnd\" must be a string",
#         "\"unitType\" is not allowed to be empty",
#         "\"issuance.orgUid\" is not allowed",
#         "\"verificationBody\" is not allowed"
#     ],
#     "success": false
# }



,originProjectId,projectName,program,sector,projectType,projectStatus,methodology,validationBody,validationDate,inCountryRegion,...,registryOfOrigin,unitMetric,projectLink,projectId,coveredByNDC,projectStatusDate,projectDeveloper,geographicIdentifier,description,ndcInformation
24,51Z7LD1J86IUVYIUFF42N3NZ6F6NOB,e7 Bhutan Micro Hydro Power CDM Project,PA,Energy industries (renewable - / non-renewable...,Hydro,Registered,CDM - AMS-I.A.,"JACO CDM CO., LTD",2004-06-04 00:00:00,Southern Asia,...,CDM Registry,tCO2e,https://cdm.unfccc.int,51Z7LD1J86IUVYIUFF42N3NZ6F6NOB,Unknown,2023-12-08,"E7, Kansai Electric","{""latitude"":27.514162,""longitude"":90.433601}",e7 Bhutan Micro Hydro Power CDM Project,NA
2254,1QA3P4SHFQ3HT5A1PADQD59FL9SPOC,"Dagachhu Hydropower Project, Bhutan",PA,Energy industries (renewable - / non-renewable...,Hydro,Registered,CDM - ACM0002,Det Norske Veritas- CUK,2008-09-03 00:00:00,Southern Asia,...,CDM Registry,tCO2e,https://cdm.unfccc.int,1QA3P4SHFQ3HT5A1PADQD59FL9SPOC,Unknown,2023-12-08,Pöyry Energy,"{""latitude"":27.514162,""longitude"":90.433601}","Dagachhu Hydropower Project, Bhutan",NA
7755,U18YV4JT0CQTF3G0LQ0I4FDSCSQM2L,"Punatsangchhu-I Hydroelectric Project, Bhutan",PA,Energy industries (renewable - / non-renewable...,Hydro,Registered,CDM - ACM0002,Bureau Veritas Certification Holding SAS (BVCH),2010-11-04 00:00:00,Southern Asia,...,CDM Registry,tCO2e,https://cdm.unfccc.int,U18YV4JT0CQTF3G0LQ0I4FDSCSQM2L,Unknown,2023-12-08,PHPA,"{""latitude"":27.514162,""longitude"":90.433601}","Punatsangchhu-I Hydroelectric Project, Bhutan",NA
8406,RAF02M1RO8Q7V2WW1RPI9ZB6U3QWM2,Rural Electrification Project for Clean Energy...,PA,Energy distribution,Energy distribution,Registered,CDM - AMS-III.AW.,RWTÜV GmbH,2013-01-22 00:00:00,Southern Asia,...,CDM Registry,tCO2e,https://cdm.unfccc.int,RAF02M1RO8Q7V2WW1RPI9ZB6U3QWM2,Unknown,2023-12-08,n.a.,"{""latitude"":27.514162,""longitude"":90.433601}",Rural Electrification Project for Clean Energy...,NA
8662,AV4D7HV34I0UH7WJ6S2G3UV9LA2KQV,Mangdechhu Hydroelectric Project Authority,PA,Energy industries (renewable - / non-renewable...,Hydro,Registered,CDM - ACM0002,Bureau Veritas Certification Holding SAS (BVCH),2014-06-26 00:00:00,Southern Asia,...,CDM Registry,tCO2e,https://cdm.unfccc.int,AV4D7HV34I0UH7WJ6S2G3UV9LA2KQV,Unknown,2023-12-08,n.a.,"{""latitude"":27.514162,""longitude"":90.433601}",Mangdechhu Hydroelectric Project Authority,NA
8977,976VA2GJENO0ZMVYWRQJ8XOPG2JXB5,Punatsangchhu-II Hydroelectric Project (1020 MW),PA,Energy industries (renewable - / non-renewable...,Hydro,Validated,CDM - ACM0002,Bureau Veritas Certification Holding SAS (BVCH),2014-06-26 00:00:00,Southern Asia,...,CDM Registry,tCO2e,https://cdm.unfccc.int,976VA2GJENO0ZMVYWRQJ8XOPG2JXB5,Unknown,2023-12-08,n.a.,"{""latitude"":27.514162,""longitude"":90.433601}",Punatsangchhu-II Hydroelectric Project (1020 MW),NA
9077,Z2GXVTKHQJ8VB82DHT5I70LG7DMABM,Substitution of grid power generation through ...,PA,Energy industries (renewable - / non-renewable...,Hydro,Withdrawn,CDM - ACM0002,Det Norske Veritas- CUK,2007-12-22 00:00:00,Southern Asia,...,CDM Registry,tCO2e,https://cdm.unfccc.int,Z2GXVTKHQJ8VB82DHT5I70LG7DMABM,Unknown,2023-12-08,Power Grid Corporation of India,"{""latitude"":27.514162,""longitude"":90.433601}",Substitution of grid power generation through ...,NA
12038,YDCO88MCSYRWQVM9SXIAN4ZVD0TFF0,"Dagachhu Hydropower Project, Bhutan",PA,Energy industries (renewable - / non-renewable...,Hydro,Validated,CDM - ACM0002,Det Norske Veritas- CUK,2007-10-23 00:00:00,Southern Asia,...,CDM Registry,tCO2e,https://cdm.unfccc.int,YDCO88MCSYRWQVM9SXIAN4ZVD0TFF0,Unknown,2023-12-08,"Caspervandertak, Gansu Tonghe Investment Proje...","{""latitude"":27.514162,""longitude"":90.433601}","Dagachhu Hydropower Project, Bhutan",NA
